<a href="https://colab.research.google.com/github/bhavya-kashyap/AMC_crosswalk/blob/main/abstract_pubmed_webscrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install bs4
!pip install pymed

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://securit

In [2]:
import pandas as pd
import numpy as np
import pymed
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import math

In [3]:
# Data parameters
# https://www.webofscience.com/wos/woscc/summary/f05a3dd0-ac52-4364-859d-aea68cf4f4d3-583fd5ef/relevance/1
# SO = (MMWR RECOMMENDATIONS AND REPORTS OR NEW ENGLAND JOURNAL OF MEDICINE OR NATURE MEDICINE OR NATURE REVIEWS IMMUNOLOGY OR MORBIDITY AND MORTALITY WEEKLY REPORT OR LANCET OR PHYSIOLOGICAL REVIEWS OR NATURE REVIEWS DRUG DISCOVERY OR ANNUAL REVIEW OF PATHOLOGY: MECHANISMS OF DISEASE OR JOURNAL OF CLINICAL ONCOLOGY OR LANCET GLOBAL HEALTH OR ANNALS OF ONCOLOGY OR ANNUAL REVIEW OF PUBLIC HEALTH OR NATURE REVIEWS DISEASE PRIMERS OR ACCOUNTS OF CHEMICAL RESEARCH OR ANNUAL REVIEW OF CLINICAL PSYCHOLOGY OR JOURNAL OF EXPERIMENTAL MEDICINE OR MOLECULAR SYSTEMS BIOLOGY OR ANNUAL REVIEW OF MICROBIOLOGY OR JAMA - JOURNAL OF THE AMERICAN MEDICAL ASSOCIATION OR LANCET DIGITAL HEALTH OR REPORTS ON PROGRESS IN PHYSICS OR NATURE BIOMEDICAL ENGINEERING OR EMBO JOURNAL OR JAMA PSYCHIATRY OR NATURE REVIEWS MOLECULAR CELL BIOLOGY OR CELL OR NATURE REVIEWS GENETICS OR NATURE BIOTECHNOLOGY OR NATURE REVIEWS CANCER OR NATURE GENETICS OR ANNUAL REVIEW OF BIOCHEMISTRY OR NATURE CATALYSIS OR NATURE METHODS OR CANCER CELL OR MOLECULAR CELL OR NATURE CELL BIOLOGY OR LANCET DIABETES AND ENDOCRINOLOGY OR NATURE MICROBIOLOGY OR ANNUAL REVIEW OF PLANT BIOLOGY OR ANNUAL REVIEW OF GENETICS OR CELL HOST AND MICROBE OR GENOME BIOLOGY OR CELL METABOLISM OR NATURE REVIEWS ENDOCRINOLOGY OR NATURE STRUCTURAL AND MOLECULAR BIOLOGY OR NUCLEIC ACIDS RESEARCH OR PHARMACOLOGICAL REVIEWS OR ANNUAL REVIEW OF PHARMACOLOGY AND TOXICOLOGY OR NANO TODAY OR DRUG RESISTANCE UPDATES OR JOURNAL FOR IMMUNOTHERAPY OF CANCER OR TRENDS IN PHARMACOLOGICAL SCIENCES OR EMERGING MICROBES AND INFECTIONS OR PHARMACOLOGY AND THERAPEUTICS OR MOLECULAR THERAPY OR NPJ VACCINES OR ADVANCED DRUG DELIVERY REVIEWS OR NATURAL PRODUCT REPORTS OR CELL CHEMICAL BIOLOGY OR PROTEIN AND CELL OR CELLULAR AND MOLECULAR LIFE SCIENCES OR MEDICINAL RESEARCH REVIEWS OR NEUROPSYCHOPHARMACOLOGY OR ANTIVIRAL RESEARCH OR ACTA PHARMACEUTICA SINICA B OR PAIN OR THERANOSTICS OR ADVANCED HEALTHCARE MATERIALS OR BRITISH JOURNAL OF PHARMACOLOGY OR ENVIRONMENTAL POLLUTION OR ALZHEIMER'S AND DEMENTIA OR HEALTH AFFAIRS OR LANCET PLANETARY HEALTH OR IMPLEMENTATION SCIENCE OR INFECTIOUS DISEASE MODELLING OR BMJ GLOBAL HEALTH OR GLOBALIZATION AND HEALTH OR JOURNAL OF HEALTH ECONOMICS OR JOURNAL OF THE AMERICAN MEDICAL DIRECTORS ASSOCIATION OR MILBANK QUARTERLY OR EUROPEAN HEART JOURNAL. QUALITY OF CARE &AMP, CLINICAL OUTCOMES OR BMJ QUALITY AND SAFETY OR HEALTH SERVICES RESEARCH OR MEDICAL CARE RESEARCH AND REVIEW OR VALUE IN HEALTH OR POLICY AND INTERNET OR INTERNATIONAL JOURNAL OF DRUG POLICY OR INTERNATIONAL JOURNAL FOR EQUITY IN HEALTH OR JOURNAL OF GLOBAL HEALTH OR PHARMACOECONOMICS OR HEALTH RESEARCH POLICY AND SYSTEMS OR GLOBAL HEALTH RESEARCH AND POLICY OR INTERNATIONAL JOURNAL OF TRANSGENDER HEALTH OR JCO ONCOLOGY PRACTICE OR PREVENTING CHRONIC DISEASE OR NATURE OR SCIENCE OR SCIENCE ADVANCES OR PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES OF THE UNITED STATES OF AMERICA OR NATIONAL SCIENCE REVIEW OR SCIENCE BULLETIN OR RESEARCH OR JOURNAL OF ADVANCED RESEARCH OR ISCIENCE OR SCIENTIFIC REPORTS OR TSINGHUA SCIENCE AND TECHNOLOGY OR PROCEEDINGS OF THE JAPAN ACADEMY SERIES B: PHYSICAL AND BIOLOGICAL SCIENCES OR ADVANCED THEORY AND SIMULATIONS OR PROGRESS IN NATURAL SCIENCE: MATERIALS INTERNATIONAL OR PLOS ONE OR SCIENCE AND TECHNOLOGY STUDIES OR ROYAL SOCIETY OPEN SCIENCE OR RESEARCH IN SCIENCE AND TECHNOLOGICAL EDUCATION OR INTERNATIONAL JOURNAL OF BIFURCATION AND CHAOS IN APPLIED SCIENCES AND ENGINEERING OR LATIN AMERICAN RESEARCH REVIEW OR FRACTALS OR FACETS OR HELIYON OR JOURNAL OF KING SAUD UNIVERSITY - SCIENCE OR ARABIAN JOURNAL FOR SCIENCE AND ENGINEERING OR CLINICAL AND TRANSLATIONAL IMMUNOLOGY OR JOURNAL OF PAIN AND SYMPTOM MANAGEMENT OR INTERNATIONAL JOURNAL OF NURSING STUDIES OR NURSE EDUCATION TODAY OR JOURNAL OF NURSING SCHOLARSHIP OR JOURNAL OF PALLIATIVE MEDICINE OR PATIENT OR CLINICAL SIMULATION IN NURSING OR INTERNATIONAL NURSING REVIEW OR JOURNAL OF CLINICAL NURSING OR WORLDVIEWS ON EVIDENCE-BASED NURSING OR NURSE EDUCATION IN PRACTICE OR JOURNAL OF ADVANCED NURSING OR JOURNAL OF SCHOOL NURSING OR JOURNAL OF RELIGION AND HEALTH OR INTERNATIONAL JOURNAL OF NURSING SCIENCES OR NURSING OUTLOOK OR PALLIATIVE AND SUPPORTIVE CARE OR SYMBOLIC INTERACTION OR JOURNAL OF PROFESSIONAL NURSING OR JOURNAL OF TRANSCULTURAL NURSING OR JOURNAL OF NURSING EDUCATION OR BMC NURSING OR JOURNAL OF MULTIDISCIPLINARY HEALTHCARE)
# Publication years: last 5 years (2018 - 2022)
# Affiliations: Emory university
# Size of data: 1000 records
# primary focus is to get the pubmed ID

In [7]:
df = pd.read_csv("sampledata.csv")
df.shape

In [9]:
# df.head()
df.drop(columns = ['Publication Type','Authors','Author Full Names', 'Group Authors','Source Title','Document Type', 'Publication Date','Publication Year', 'Issue','Start Page','End Page', 'Article Number','DOI','Volume', 'DOI Link', 'Early Access Date', 'UT (Unique WOS ID)', 'Web of Science Record'], inplace=True)

In [10]:
baseurl = 'https://pubmed.ncbi.nlm.nih.gov/'
urls = []
for i in range(len(df)):
    urls.append(baseurl + str(df.loc[i]['Pubmed Id'].astype(int)) + '/')
df['URL'] = urls
df.head(10)

,Article Title,Abstract,Pubmed Id,URL
0,A low-cost genomics workflow enables isolate s...,Earth's environments harbor complex consortia ...,36224660.0,https://pubmed.ncbi.nlm.nih.gov/36224660/
1,A saturated map of common genetic variants ass...,Common single-nucleotide polymorphisms (SNPs) ...,36224396.0,https://pubmed.ncbi.nlm.nih.gov/36224396/
2,Evaluation of a non-pharmacological interdisci...,Objective To conduct a quality improvement eva...,36064854.0,https://pubmed.ncbi.nlm.nih.gov/36064854/
3,Comparative genomic and biochemical analyses i...,Humans and Acanthamoeba polyphaga mimivirus sh...,36207453.0,https://pubmed.ncbi.nlm.nih.gov/36207453/
4,Africa must participate in finding a gene ther...,NaN,36203001.0,https://pubmed.ncbi.nlm.nih.gov/36203001/
5,ADHESION G PROTEIN-COUPLED RECEPTORS: STRUCTUR...,Adhesion G protein-coupled receptors (AGPCRs) ...,35468004.0,https://pubmed.ncbi.nlm.nih.gov/35468004/
6,Authorship equity guidelines in global health ...,NaN,36220309.0,https://pubmed.ncbi.nlm.nih.gov/36220309/
7,Stroke genetics informs drug discovery and ris...,Previous genome-wide association studies (GWAS...,36180795.0,https://pubmed.ncbi.nlm.nih.gov/36180795/
8,Toward a Uniform Classification of Nurse Pract...,Many states' scope of practice laws limits the...,36172783.0,https://pubmed.ncbi.nlm.nih.gov/36172783/
9,p38MAPK guards the integrity of endosomal comp...,Pathogens trigger activation of sensors of the...,36175595.0,https://pubmed.ncbi.nlm.nih.gov/36175595/


In [11]:
sample = df.head(20)

In [12]:
def driversetup():
      options = webdriver.ChromeOptions()
      #run Selenium in headless mode
      options.add_argument('--headless')
      options.add_argument('--no-sandbox')
      options.add_argument("enable-automation")
      #overcome limited resource problems
      options.add_argument('--disable-dev-shm-usage')
      options.add_argument("lang=en")
      #open Browser in maximized mode
      options.add_argument("start-maximized")
      #disable infobars
      options.add_argument("disable-infobars")
      #disable extension
      options.add_argument("--disable-browser-side-navigation")
      options.add_argument("--disable-gpu")
      options.add_argument("--disable-extensions")
      options.add_argument("--incognito")
      options.add_argument("--disable-blink-features=AutomationControlled")
      driver = webdriver.Chrome(options=options)
      driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")
      return driver

def pagesource(url, driver):
      driver = driver
      driver.get(url)
      time.sleep(8)
      soup = BeautifulSoup(driver.page_source)
      return soup

In [14]:
d = driversetup()
abstracts = []
for i in range(len(sample)):
  s = pagesource(sample.loc[i]['URL'], d)
  abstract = list(s.findAll('div', class_="abstract-content selected"))
  a = ''
  for j in range(len(abstract)):
    a = a + ' ' + abstract[j].text
  abstracts.append(a)
  print(i, a)
sample["PubMed_Abstracts"] = abstracts

0  

      
      Earth's environments harbor complex consortia of microbes that affect processes ranging from host health to biogeochemical cycles. Understanding their evolution and function is limited by an inability to isolate genomes in a high-throughput manner. Here, we present a workflow for bacterial whole-genome sequencing using open-source labware and the OpenTrons robotics platform, reducing costs to approximately $10 per genome. We assess genomic diversity within 45 gut bacterial species from wild-living chimpanzees and bonobos. We quantify intraspecific genomic diversity and reveal divergence of homologous plasmids between hosts. This enables population genetic analyses of bacterial strains not currently possible with metagenomic data alone.
    

1  

      
      Common single-nucleotide polymorphisms (SNPs) are predicted to collectively explain 40-50% of phenotypic variation in human height, but identifying the specific variants and associated regions requires huge sampl

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [15]:
sample

,Article Title,Abstract,Pubmed Id,URL,PubMed_Abstracts
0,A low-cost genomics workflow enables isolate s...,Earth's environments harbor complex consortia ...,36224660.0,https://pubmed.ncbi.nlm.nih.gov/36224660/,\n\n \n Earth's environments harbor...
1,A saturated map of common genetic variants ass...,Common single-nucleotide polymorphisms (SNPs) ...,36224396.0,https://pubmed.ncbi.nlm.nih.gov/36224396/,\n\n \n Common single-nucleotide po...
2,Evaluation of a non-pharmacological interdisci...,Objective To conduct a quality improvement eva...,36064854.0,https://pubmed.ncbi.nlm.nih.gov/36064854/,\n\n\n Objective:\n \n \...
3,Comparative genomic and biochemical analyses i...,Humans and Acanthamoeba polyphaga mimivirus sh...,36207453.0,https://pubmed.ncbi.nlm.nih.gov/36207453/,\n\n \n Humans and Acanthamoeba pol...
4,Africa must participate in finding a gene ther...,NaN,36203001.0,https://pubmed.ncbi.nlm.nih.gov/36203001/,
5,ADHESION G PROTEIN-COUPLED RECEPTORS: STRUCTUR...,Adhesion G protein-coupled receptors (AGPCRs) ...,35468004.0,https://pubmed.ncbi.nlm.nih.gov/35468004/,\n\n \n Adhesion G protein-coupled ...
6,Authorship equity guidelines in global health ...,NaN,36220309.0,https://pubmed.ncbi.nlm.nih.gov/36220309/,
7,Stroke genetics informs drug discovery and ris...,Previous genome-wide association studies (GWAS...,36180795.0,https://pubmed.ncbi.nlm.nih.gov/36180795/,\n\n \n Previous genome-wide associ...
8,Toward a Uniform Classification of Nurse Pract...,Many states' scope of practice laws limits the...,36172783.0,https://pubmed.ncbi.nlm.nih.gov/36172783/,\n\n \n Many states' scope of pract...
9,p38MAPK guards the integrity of endosomal comp...,Pathogens trigger activation of sensors of the...,36175595.0,https://pubmed.ncbi.nlm.nih.gov/36175595/,\n\n \n Pathogens trigger activatio...
